In [25]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import pandas as pd
from datetime import datetime
import csv

# Test, traitement, récupération des traces

## Traces des messages

In [26]:
data_1 = pd.read_json('Data\messages_chat_seance1\\0dfd43cb-9957-4102-954a-899ca075e56d.json')
data_2 = pd.read_json('Data\messages_chat_seance1\\1db9dee2-2702-457f-aa07-6b60589446ce.json')
data_3 = pd.read_json('Data\messages_chat_seance1\\2ccf746f-62b9-4690-8bff-59c49fb2c310.json')

Recupérer ici la fin du fichier : seance 1 ne regroupe que 3 messages :: mettre dans seance 1 tous les messages de la seance 1
creer csv avec toutes les données : seance + nomfichier comme id.

In [27]:
#Gros df avec les 3 données
messages_df = pd.concat([data_1,data_2,data_3],axis = 0)
messages_df.reset_index(drop=True, inplace=True)
messages_df.head(5)

,user,role,message,timestamp
0,194872,Driver,roro,2024-10-09 04:24:09.074000+00:00
1,194872,Driver,Qui est là,2024-10-09 04:27:03.939000+00:00
2,145797,Driver,Bonjour,2024-10-09 04:22:52.924000+00:00
3,200807,Navigator,Bonjour,2024-10-09 04:24:14.448000+00:00
4,145797,Driver,tu pense si je fais un for in in range avec un...,2024-10-09 04:25:31.692000+00:00


In [28]:
##Fonction(s) pour traiter les messages

def parse_json_files_in_folder(folder_path):
    """
    Parses all JSON files in a folder and concatenates them into a pandas DataFrame.

    Args:
        folder_path (str): The path to the folder containing the JSON files.

    Returns:
        pandas.DataFrame: A DataFrame containing the concatenated data from all JSON files.
    """
    all_data = pd.DataFrame()
    groupid = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            filepath = os.path.join(folder_path, filename)
            data = pd.read_json(filepath)
            data['groupID'] = groupid
            data['fileID'] = filename[:-5]
            all_data = pd.concat([all_data,data],axis = 0)
            groupid += 1
    all_data.reset_index(inplace = True, drop = True)
    all_data.index.names = ["messageID"]
    # extracting date from timestamp
    all_data['date'] = [datetime.date(d) for d in all_data['timestamp']] 

    # extracting time from timestamp
    all_data['time'] = [datetime.time(d).replace(microsecond=0) for d in all_data['timestamp']]
    all_data.drop(['timestamp'],axis = 1,inplace = True)
    return all_data


In [29]:
seance1_df = parse_json_files_in_folder('Data\messages_chat_seance1')
print("seance1_df shape : ",seance1_df.shape)
seance1_df

seance1_df shape :  (451, 7)


,user,role,message,groupID,fileID,date,time
messageID,,,,,,,
0,186202.0,Navigator,"Tu es qui ,",0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:10
1,186202.0,Navigator,On se met 5 partout ok ?,0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:23
2,183897.0,Driver,Opé,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:22
3,183897.0,Driver,Opé lmkt,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:39
4,193392.0,Navigator,A PAR SA LE COUZ,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:46
...,...,...,...,...,...,...,...
446,188437.0,Navigator,tu as fais des tests?,49,f374e410-f777-4ef0-8a15-14267dd8ed09,2024-10-09,04:28:34
447,200552.0,Driver,pas encore,49,f374e410-f777-4ef0-8a15-14267dd8ed09,2024-10-09,04:28:42
448,188437.0,Navigator,ça l'air correct,49,f374e410-f777-4ef0-8a15-14267dd8ed09,2024-10-09,04:29:16


In [30]:
seance2_df = parse_json_files_in_folder('Data\messages_chat_seance2')
print("seance2_df shape : ",seance2_df.shape)
seance2_df

seance2_df shape :  (1315, 7)


,groupID,fileID,user,role,message,date,time
messageID,,,,,,,
0,1,00c09563-99f4-42da-9346-d73355a9b8d9,183337.0,Driver,bonjour,2024-11-12,04:30:24
1,1,00c09563-99f4-42da-9346-d73355a9b8d9,145133.0,Navigator,bonjour,2024-11-12,04:30:44
2,1,00c09563-99f4-42da-9346-d73355a9b8d9,145133.0,Navigator,tu es bloqué?,2024-11-12,04:36:03
3,1,00c09563-99f4-42da-9346-d73355a9b8d9,145133.0,Driver,hésite pas si t'as une idée,2024-11-12,04:39:50
4,1,00c09563-99f4-42da-9346-d73355a9b8d9,183337.0,Navigator,ok,2024-11-12,04:40:06
...,...,...,...,...,...,...,...
1310,81,fbe22b37-299a-4851-a4a0-c2f152bc835f,144245.0,Driver,c'est bon,2024-11-12,04:48:02
1311,81,fbe22b37-299a-4851-a4a0-c2f152bc835f,192807.0,Navigator,"A la base, ce n'est pas total non ?",2024-11-12,04:48:09
1312,81,fbe22b37-299a-4851-a4a0-c2f152bc835f,192807.0,Navigator,Tu voulais tous les mettre total non ?,2024-11-12,04:49:41


In [31]:
#Ajout d'une séance colonne pour différencier les séances
seance1_df['seanceID'] = 1
seance1_df.head(5)

,user,role,message,groupID,fileID,date,time,seanceID
messageID,,,,,,,,
0,186202.0,Navigator,"Tu es qui ,",0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:10,1
1,186202.0,Navigator,On se met 5 partout ok ?,0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:23,1
2,183897.0,Driver,Opé,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:22,1
3,183897.0,Driver,Opé lmkt,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:39,1
4,193392.0,Navigator,A PAR SA LE COUZ,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:46,1


In [32]:
seance2_df['seanceID'] = 2
seance2_df.head(5)

,groupID,fileID,user,role,message,date,time,seanceID
messageID,,,,,,,,
0,1,00c09563-99f4-42da-9346-d73355a9b8d9,183337.0,Driver,bonjour,2024-11-12,04:30:24,2
1,1,00c09563-99f4-42da-9346-d73355a9b8d9,145133.0,Navigator,bonjour,2024-11-12,04:30:44,2
2,1,00c09563-99f4-42da-9346-d73355a9b8d9,145133.0,Navigator,tu es bloqué?,2024-11-12,04:36:03,2
3,1,00c09563-99f4-42da-9346-d73355a9b8d9,145133.0,Driver,hésite pas si t'as une idée,2024-11-12,04:39:50,2
4,1,00c09563-99f4-42da-9346-d73355a9b8d9,183337.0,Navigator,ok,2024-11-12,04:40:06,2


In [33]:
all_messages_df = pd.concat([seance1_df,seance2_df],axis = 0,ignore_index = True)
all_messages_df.index.names=['messageID']
#print(all_messages_df.head(5),all_messages_df.tail(5))
all_messages_df

,user,role,message,groupID,fileID,date,time,seanceID
messageID,,,,,,,,
0,186202.0,Navigator,"Tu es qui ,",0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:10,1
1,186202.0,Navigator,On se met 5 partout ok ?,0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:23,1
2,183897.0,Driver,Opé,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:22,1
3,183897.0,Driver,Opé lmkt,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:39,1
4,193392.0,Navigator,A PAR SA LE COUZ,1,0435458c-dc49-4094-b970-1db125fc235c,2024-10-09,04:21:46,1
...,...,...,...,...,...,...,...,...
1761,144245.0,Driver,c'est bon,81,fbe22b37-299a-4851-a4a0-c2f152bc835f,2024-11-12,04:48:02,2
1762,192807.0,Navigator,"A la base, ce n'est pas total non ?",81,fbe22b37-299a-4851-a4a0-c2f152bc835f,2024-11-12,04:48:09,2
1763,192807.0,Navigator,Tu voulais tous les mettre total non ?,81,fbe22b37-299a-4851-a4a0-c2f152bc835f,2024-11-12,04:49:41,2


In [34]:
all_messages_df.to_csv('all_messages.csv',quoting=csv.QUOTE_NONNUMERIC) #"Quote_nonnumeric" permet de quote le texte et de ne pas avoir de pb avec les virgules

## Traces complètes

In [35]:
def get_data_from_file(filepath):
    f = open(filepath,'r')
    trace = json.load(f)
    f.close()
    df = pd.DataFrame(columns = ['actor','role','type','message','timestamp'])

    for i in range(len(trace['statements'])):
        if trace['statements'][i]['verb']['id']!="Connected" and trace['statements'][i]['verb']['id']!= "Disconnected" and trace['statements'][i]['verb']['id'] !='PairedWith':
            if trace['statements'][i]['verb']['id'] == "SentInChat":
                df.loc[len(df)] = [trace['statements'][i]['actor']['account'],'',trace['statements'][i]['verb']['id'],trace['statements'][i]['context']['extensions']['chatMessageSent']['message'],trace['statements'][i]['timestamp']['$date']]
            elif trace['statements'][i]['verb']['id'] == "Wrote":
                df.loc[len(df)] = [trace['statements'][i]['actor']['account'],'',trace['statements'][i]['verb']['id'],trace['statements'][i]['object']['definition']['extensions']['codeSnapshot']['code'],trace['statements'][i]['timestamp']['$date']]
            else:
                df.loc[len(df)] = [trace['statements'][i]['actor']['account'],'',trace['statements'][i]['verb']['id'],'',trace['statements'][i]['timestamp']['$date']]
                #Decommenter pour stopper la lecture à "EndedActivity"
                #if trace['statements'][i]['verb']['id'] == "EndedActivity":
                    #return df
    return df

def get_data_from_folder(folder_path):
    file_data = pd.DataFrame()
    groupid = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            filepath = os.path.join(folder_path, filename)
            #print(filepath)
            data = get_data_from_file(filepath)
            data['groupID'] = groupid
            data['fileID'] = filename[:-5]
            file_data = pd.concat([file_data,data],axis = 0)
            groupid += 1
    file_data.reset_index(inplace = True, drop = True)
    file_data.index.names = ["messageID"]
    # extracting date from timestamp
    file_data['date'] = [datetime.date(datetime.strptime(d,'%Y-%m-%dT%H:%M:%S.%fZ')) for d in file_data['timestamp']] 

    # extracting time from timestamp
    file_data['time'] = [datetime.time(datetime.strptime(d,'%Y-%m-%dT%H:%M:%S.%fZ')).replace(microsecond=0) for d in file_data['timestamp']]
    file_data.drop(['timestamp'],axis = 1,inplace = True)
    return file_data


In [36]:
all_data_s1 = get_data_from_folder('Data\\traces_seance1')
all_data_s2 = get_data_from_folder('Data\\traces_seance2')

In [37]:
all_data = pd.concat([all_data_s1,all_data_s2],axis=0,ignore_index=True)
all_data.index.names=['dataID']
all_data

,actor,role,type,message,groupID,fileID,date,time
dataID,,,,,,,,
0,189732,,Wrote,,0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:00
1,189732,,EndedActivity,,0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:00
2,186202,,SentInChat,"Tu es qui ,",0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:10
3,186202,,SentInChat,On se met 5 partout ok ?,0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:38:23
4,189732,,Wrote,,0,023a3090-7b0a-48c4-93f0-c200d9afce41,2024-10-09,04:39:24
...,...,...,...,...,...,...,...,...
4483,192807,,EndedActivity,,107,fbe22b37-299a-4851-a4a0-c2f152bc835f,2024-11-12,05:06:17
4484,144245,,Wrote,"VALEUR = {""I"" : 1, ""V"" : 5, ""X"" : 10, ""L"" : 50...",107,fbe22b37-299a-4851-a4a0-c2f152bc835f,2024-11-12,05:08:37
4485,144245,,SentInChat,c'est bon,107,fbe22b37-299a-4851-a4a0-c2f152bc835f,2024-11-12,05:11:26


In [ ]:
all_data.to_csv('all_data.csv')